In [8]:
from astropy.io import fits
from astropy import wcs
import bz2
import os
import urllib.request
import numpy as np
import copy
import matplotlib.pyplot as plt
import csv

In [9]:
def asymmetry(ra, dec, r, petrorad, expab, expphi, run, camcol, field, rowf, colf, FITS_directory='Fits files/'  ,rerun=301):
    #Returns an array of the fourier A1 parameter and the phase angle for a given object in the SDSS field specified
    #by run, rerun, camcol, field, row, col @ row, col.
    #r= Array of Radiuss to probe (in number of petrosian radiuses) to calculate the fourier coefficient for
    #petrorad=the petrosian radius of the object in arcseconds
    #expphi = The angle of the exponential fit, expressed as degrees, N through E
    #expAB= The ratio of the semi major axes of the disk
    # Run, rerun, camcol, field = Specified the SDSS field the object is contained in
    # rowf, colf = the pixel coordinates of the centre of the image, in FITS coordinates (ie, first pixel is 1, not zero) 
    # FITS_directory=Location of folder of fits files
    #Open FITS file from disk
    try:
        #Generate the name the FITS file will be stored under
        runstring='0'*(6-len(str(run)))+str(run) 
        fieldstring='0'*(4-len(str(field)))+str(field)
        fitsname='frame-i-'+runstring+'-'+str(camcol)+'-'+fieldstring
        #Try to open the fits file. If it throws a filenotfound error, it's because this field hasn't been downloaded yet
        file=fits.open(FITS_directory+fitsname+'.fits')
    except FileNotFoundError:
        #Generate the URL the zipped FITS file will be stored under
        fitsurl='http://data.sdss3.org/sas/dr12/boss/photoObj/frames/'+str(rerun)+'/'+str(run)+'/'+str(camcol)+'/'+fitsname+'.fits.bz2'
        print('Downloading from ', fitsurl)
        #Save the zipped FITS file, extract the unzipped fits file
        urllib.request.urlretrieve(fitsurl, FITS_directory+fitsname+'.fits.bz2')
        f=bz2.open(FITS_directory+fitsname+'.fits.bz2')
        #Save the unzipped FITS file to disk
        save=fits.open(f)
        save.writeto(FITS_directory+fitsname+'.fits')
        save.close()
        #Remove the zipped file to save space
        os.remove(FITS_directory+fitsname+'.fits.bz2')
        file=fits.open(FITS_directory+fitsname+'.fits')
    image=file[0] 
    data=image.data
    output=[]
    
    #If user has entered single radius, converts this to list so can be iterated over later
    if type(r)==float or type(r)==int:
        r=[r]
    #Convert centre row and column to python convention (starting at zero), instead of FITS convention(starting at 1)
    rowc=rowf-1
    colc=colf-1
    
    
    #Get coordinate parameters from header using astropy WCS class. 
    coords=wcs.WCS(image.header)
    
    #Checks ra & dec agree with FITS header to within 1 pixel. 
    checkradec(ra, dec, coords, rowc, colc)
    
    #Set up sub-function to return row, col pairs for given ra, dec values
    def pix(raf,decf):
        #Takes values of ra and dec (in degrees). Converts to an array of [row, col] in pixel values
        array=coords.all_world2pix(raf,decf,0)
        return array[::-1]
    

    #Get length of semi-major and semi-minor axes of petrosian ellipse. Assume petrosian radius is the geometric mean of a & b. In degrees
    a=(2./(1+expab**2))**0.5*petrorad/3600.
    b=(2./(1+expab**-2))**0.5*petrorad/3600.
    
    
   
    
    
    #Create an array of theta values to sample the ellipse at
    dtheta=360./360.
    thetas=np.arange(0,360,dtheta)
    thetarad=[i*2*np.pi/360. for i in thetas]
    for relement in r: 
        #Get semi-major and semi-minor axes of the ellipse to calculate the fourier coefficients around. In arcsec
        ar=a*relement
        br=b*relement
        rs=getrs(thetas, ar, br, expphi)
        #Create array with data values for each theta, r combination
        pixbrightness=[]
        for i in range(len(thetas)):
            #Calculate ra, dec values of target
            #First calculate distances from centre
            drai=rs[i]*np.sin(thetarad[i])
            ddeci=rs[i]*np.cos(thetarad[i])
            #Then add on coordinates of centre
            rai=ra+drai
            deci=dec+ddeci

            #Convert to pixels
            pixel=pix(rai,deci)
            rowi=float(pixel[0])
            coli=float(pixel[1])
            rowiround=int(round(rowi))
            coliround=int(round(coli))
            pixbrightness.append(data[rowiround, coliround])


        #Weight the pixel brightnesses by cos(theta) and sin(theta)
        brightnesssin=[pixbrightness[i]*np.sin(thetarad[i]) for i in range(len(thetarad))]
        brightnesscos=[pixbrightness[i]*np.cos(thetarad[i]) for i in range(len(thetarad))]
        #Find the a1 & b1 fourier coefficients. 1/pi * integral over pixelbrighness sin/cos (integrated using np.trapz). f(x)=ancosntheta+ bnsinntheta, summed over all n
        a1=np.pi**-1*np.trapz(brightnesscos, thetarad)
        b1=np.pi**-1*np.trapz(brightnesssin, thetarad)
        a0=np.pi**-1*np.trapz(pixbrightness, thetarad)
        #Now find fourier coefficients in the form f(x)=Ancos(ntheta-phi)
        A1=(a1**2+b1**2)**0.5
        phi=np.arctan(-a1/b1)
        output.append([A1, phi*360./2/np.pi, A1/a0, a0])
    return output
    file.close()
    
    
    
def ellipse(a,b,theta,phi):
    #returns the r coordinate of the ellipse with:
    #semi major axes a,b at polar angle theta degrees (measured from north, through east),
    #with semi-major axis offset from north by phi degrees (measured from north through east)
    thetarad=theta*2*np.pi/360.
    phirad=phi*2*np.pi/360.
    r=a*b/((b*np.cos(thetarad-phirad))**2+(a*np.sin(thetarad-phirad))**2)**0.5
    return r

def checkradec(ra, dec, coords, row, col):
    #checks to see if the pixel coordinates returned by a WCS object coords match ra and dec to correct values row and column
    reportedrow=coords.all_world2pix(ra,dec,0)[1]
    reportedcol=coords.all_world2pix(ra,dec,0)[0]
    try:
        if (reportedcol-col)**2>1 or (reportedrow-row)**2>1:
            raise ValueError('Coordinates from PhotoObj and ra & dec values do not match. Row discrepancy is ', reportedrow-row, 'Column discrepancy is', reportedcol-col)
    except ValueError:
        pass
def getrs(thetas, a, b, phi):
    rs=[]
    for i in thetas:
        rs.append(ellipse(a,b,i,phi))
    return rs
def functionchecker(ra, dec, r, petrorad, expab, expphi, run, camcol, field, rowf, colf, A1, A0, phi, FITS_directory='Fits files/'  ,rerun=301, A2=0.2, A3=0.14):
    #Opens the fits file for the given galaxy coordinates, then edits it to make the specific fourier components A1 and phi at radius r petrorads
       #Creates a fits file with an ellipse drawn on at r petrosian radii for a given object in the SDSS field specified
    #by run, rerun, camcol, field, row, col @ row, col.
    #r= Radius (in number of petrosian radiuses) to calculate the fourier coefficient for
    #petrorad=the petrosian radius of the object in arcseconds
    #expphi = The angle of the exponential fit, expressed as degrees, N through E
    #expAB= The ratio of the semi major axes of the disk
    # Run, rerun, camcol, field = Specified the SDSS field the object is contained in
    # rowf, colf = the pixel coordinates of the centre of the image, in FITS coordinates (ie, first pixel is 1, not zero) 
    #A1, A0, phi(degrees) - first fourier coefficients
    # FITS_directory=Location of folder of fits files
    #Open FITS file from disk
    try:
        #Generate the name the FITS file will be stored under
        runstring='0'*(6-len(str(run)))+str(run) 
        fieldstring='0'*(4-len(str(field)))+str(field)
        fitsname='frame-i-'+runstring+'-'+str(camcol)+'-'+fieldstring
        #Try to open the fits file. If it throws a filenotfound error, it's because this field hasn't been downloaded yet
        file=fits.open(FITS_directory+fitsname+'.fits')
    except FileNotFoundError:
        #Generate the URL the zipped FITS file will be stored under
        fitsurl='http://data.sdss3.org/sas/dr12/boss/photoObj/frames/'+str(rerun)+'/'+str(run)+'/'+str(camcol)+'/'+fitsname+'.fits.bz2'
        #Save the zipped FITS file, extract the unzipped fits file
        urllib.request.urlretrieve(fitsurl, FITS_directory+fitsname+'.fits.bz2')
        f=bz2.open(FITS_directory+fitsname+'.fits.bz2')
        #Save the unzipped FITS file to disk
        save=fits.open(f)
        save.writeto(FITS_directory+fitsname+'.fits')
        save.close()
        #Remove the zipped file to save space
        os.remove(FITS_directory+fitsname+'.fits.bz2')
        file=fits.open(FITS_directory+fitsname+'.fits')
    image=file[0]
    data=image.data
    
    #Convert centre row and column to python convention (starting at zero), instead of FITS convention(starting at 1)
    rowc=rowf-1
    colc=colf-1
    
    
    #Get coordinate parameters from header using astropy WCS class. 
    coords=wcs.WCS(image.header)
    
    #Checks ra & dec agree with FITS header to within 1 pixel. 
    checkradec(ra, dec, coords, rowc, colc)
    
    #Set up sub-function to return row, col pairs for given ra, dec values
    def pix(raf,decf):
        #Takes values of ra and dec (in degrees). Converts to an array of [row, col] in pixel values
        array=coords.all_world2pix(raf,decf,0)
        return array[::-1]
    

    #Get length of semi-major and semi-minor axes of petrosian ellipse. Assume petrosian radius is the geometric mean of a & b. In degrees
    a=(2./(1+expab**2))**0.5*petrorad/3600.
    b=(2./(1+expab**-2))**0.5*petrorad/3600.
    
    #Get semi-major and semi-minor axes of the ellipse to calculate the fourier coefficients around. In arcsec
    ar=a*r
    br=b*r
    
    
    #Create an array of theta values to sample the ellipse at
    dtheta=360./3600.
    thetas=np.arange(0,360,dtheta)
    rs=getrs(thetas, ar, br, expphi)
    
    def fourier(thetar, An, phi, n):
        return An*np.sin(n*(thetar-phi*2*np.pi/360))
    #Change value of each pixel in ellipse in ellipselayer
    for i in range(len(thetas)):
        thetarad=thetas[i]*2*np.pi/360.
        #Calculate ra, dec values of target
        #First calculate distances from centre
        drai=rs[i]*np.sin(thetarad)
        ddeci=rs[i]*np.cos(thetarad)
        #Then add on coordinates of centre
        rai=ra+drai
        deci=dec+ddeci
        
        #Convert to pixels
        pixel=pix(rai,deci)
        rowi=float(pixel[0])
        coli=float(pixel[1])
        rowiround=int(round(rowi))
        coliround=int(round(coli))
        #Sets corresponding pixel to value 2
        data[rowiround,coliround]=fourier(thetarad,A1,phi,1)+fourier(thetarad,A2,phi,2)+fourier(thetarad,A3,phi,3)+A0/2.
    #Adds the layer to the original fits file and saves as a new fits file with camcol G (for generated) (to differentiate between real and generated fits files)
    newfilename='frame-i-'+runstring+'-G'+'-'+fieldstring
    file.writeto(FITS_directory+newfilename+'.fits')
   
    file.close()

In [10]:
def a1a0arraycsv(filename, radius):
    #Returns a list of A1/A0 values of the galaxies contained in filename at r petrosian radii
    #CSV file must have galaxies on each row, with the following column order: ra1, dec1, ra2, dec2, objid, distance, petrorad, expad, expphi,run,rerun,camcol, field, rowc, colc
    
    #Set up lists to hold data
    ras=[]
    decs=[]
    petrorads=[]
    expabs=[]
    phis=[]
    runs=[]
    camcols=[]
    fields=[]
    rowcs=[]
    colcs=[]
    with open(filename) as csvfile:
        reader = csv.reader(csvfile)
        #Gets data on everything except 1st row
        colnames=next(reader)
        for row in reader:
            ras.append(float(row[2]))
            decs.append(float(row[3]))
            petrorads.append(float(row[6]))
            expabs.append(float(row[7]))
            phis.append(float(row[8]))
            runs.append(row[9])
            camcols.append(row[11])
            fields.append(row[12])
            rowcs.append(float(row[13]))
            colcs.append(float(row[14]))
    #Open csvwriter for results
    with open(filename+'_results','w+') as outputfile:
        writer=csv.writer(outputfile)
        writer.writerow(['ra','dec']+[k for k in radius])
        for i in range(len(ras)):
            print(i)
            try:
                asymmetryallrs=asymmetry(ra=ras[i], dec=decs[i], r=radius, petrorad=petrorads[i], expab=expabs[i], expphi=phis[i], run=runs[i], camcol=camcols[i], field=fields[i], rowf=rowcs[i], colf=colcs[i])
                row=[ras[i],decs[i]]+[asymmetryallrs[j][2] for j in range(len(asymmetryallrs))]
                writer.writerow(row)
            except ValueError as exc:
                print(exc)
                continue
            except IndexError as exc:
                #If galaxy is too close to the edge, get index error. In this case, just append the average value of the list so far(important for calculating averages over ranges of radii later)
                row=[ras[i],decs[i]]+[asymmetryallrs[j][2] for j in range(len(asymmetryallrs))]
                writer.writerow(row)
                print(exc)
                continue


In [15]:
rtoprobe=np.arange(0.05,1.3,0.01)
a1a0arraycsv('Downloads/Data_for_asymetry4_aa2811_run5.csv',rtoprobe)

0


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92


93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179


180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264


...

5429
5430
5431
5432
5433
5434
5435
5436
5437
5438
5439
5440
5441
5442
5443
5444
5445
5446
5447
5448
5449
5450
5451
5452
5453
5454
5455
5456
5457
5458
5459
5460
5461
5462
5463
5464
5465
5466
5467
5468
5469
5470
5471
5472
5473
5474
5475
5476
5477
5478
5479
5480
5481
5482
5483
5484
5485
5486
5487
5488
5489
5490
5491
5492
5493
5494
5495
5496
5497
5498
5499
5500
5501
5502
5503
5504
5505
5506
5507
5508


5509
5510
5511
5512
5513
5514
5515
5516
5517
5518
5519
5520
5521
5522
5523
5524
5525
5526
5527
5528
5529
5530
5531
5532
5533
5534
5535
5536
5537
5538
5539
5540
5541
5542
5543
5544
5545
5546
5547
5548
5549
5550
5551
5552
5553
5554
5555
5556
5557
5558
5559
5560
5561
5562
5563
5564
5565
5566
5567
5568
5569
5570
5571
5572
5573
5574
5575
5576
5577
5578
5579
5580
5581
5582
5583
5584
5585
5586
5587
5588
5589
5590
5591


5592
5593
5594
5595
5596
5597
5598
5599
index 1489 is out of bounds for axis 0 with size 1489
5600
5601
5602
5603
5604
5605
5606
5607
5608
5609
5610
5611
5612
5613
5614
5615
5616
5617
5618
5619
5620
5621
5622
5623
5624
5625
5626
5627
5628
5629
5630
5631
5632
5633
5634
5635
5636
5637
5638
5639
5640
5641
5642
5643
5644
5645
5646
5647
5648
5649
5650
5651
5652
5653
5654
5655
5656
5657
5658
5659
5660
5661
5662
5663
5664
5665
5666
5667
index 1489 is out of bounds for axis 0 with size 1489
5668
5669
5670
5671
5672
5673
5674
5675
5676
5677
5678
5679
5680


5681
5682
5683
5684
5685
5686
5687
5688
5689


KeyboardInterrupt: 

In [17]:
rtoprobe=np.arange(0.05,1.3,0.01)
a1a0arraycsv('Downloads/Offset_bar_data_for_asymmetry_aa2811.csv',rtoprobe)



0


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
index 2048 is out of bounds for axis 1 with size 2048
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65


66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162


163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255


256
257
258
259
260


261
262
263
264
265
266
267
268
269
270


In [19]:
a1a0arraycsv('Downloads/Disk_sample635_aa2811.csv', rtoprobe)
a1a0arraycsv('Downloads/Elliptical_Data_for_asymetry4_aa2811.csv', rtoprobe)

0


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


1
2
3
4
5
6
7


8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
index 2048 is out of bounds for axis 1 with size 2048
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
index 1489 is out of bounds for axis 0 with size 1489
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
index 2048 is out of bounds for axis 1 with size 2048
102
103
104
105


106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243


244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
index 1489 is out of bounds for axis 0 with size 1489
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
48